In [10]:
df<-read.csv('d:/data/iris/iris2.csv')
head(df)

sepal.length..cm.,sepal.width..cm.,petal.length..cm.,petal.width..cm.,target,Result
5.1,3.5,1.4,0.2,setosa,0
4.9,3.0,1.4,0.2,setosa,0
4.7,3.2,1.3,0.2,setosa,0
4.6,3.1,1.5,0.2,setosa,0
5.0,3.6,1.4,0.2,setosa,0
5.4,3.9,1.7,0.4,setosa,0


In [11]:
library(dplyr)
df<-df %>% select(-target)
dim(df)
head(df)

str(df)
#종속변수의 자료형을 int => factor로 변환
df$Result <- as.factor(df$Result)
str(df) #3가지 요소면 factor 처리

[1] 150   5

sepal.length..cm.,sepal.width..cm.,petal.length..cm.,petal.width..cm.,Result
5.1,3.5,1.4,0.2,0
4.9,3.0,1.4,0.2,0
4.7,3.2,1.3,0.2,0
4.6,3.1,1.5,0.2,0
5.0,3.6,1.4,0.2,0
5.4,3.9,1.7,0.4,0


'data.frame':	150 obs. of  5 variables:
 $ sepal.length..cm.: num  5.1 4.9 4.7 4.6 5 5.4 4.6 5 4.4 4.9 ...
 $ sepal.width..cm. : num  3.5 3 3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1 ...
 $ petal.length..cm.: num  1.4 1.4 1.3 1.5 1.4 1.7 1.4 1.5 1.4 1.5 ...
 $ petal.width..cm. : num  0.2 0.2 0.2 0.2 0.2 0.4 0.3 0.2 0.2 0.1 ...
 $ Result           : int  0 0 0 0 0 0 0 0 0 0 ...
'data.frame':	150 obs. of  5 variables:
 $ sepal.length..cm.: num  5.1 4.9 4.7 4.6 5 5.4 4.6 5 4.4 4.9 ...
 $ sepal.width..cm. : num  3.5 3 3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1 ...
 $ petal.length..cm.: num  1.4 1.4 1.3 1.5 1.4 1.7 1.4 1.5 1.4 1.5 ...
 $ petal.width..cm. : num  0.2 0.2 0.2 0.2 0.2 0.4 0.3 0.2 0.2 0.1 ...
 $ Result           : Factor w/ 3 levels "0","1","2": 1 1 1 1 1 1 1 1 1 1 ...


In [12]:
library(caret)
idx_train<-createDataPartition(y=df$Result,p=0.8,list=F)
train<-df[idx_train,]
X_train<-train[,-5]
y_train<-train[,5]
test<-df[-idx_train,]
X_test<-test[,-5]
y_test<-test[,5]

Loading required package: lattice
Warning message:
"package 'lattice' was built under R version 3.6.2"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.2"

In [23]:
library(e1071)
set.seed(123)
# svm 모형의 최적의 파라미터를 찾는 과정
tune.out<-tune(svm,Result ~. , data=train,
              range=list(cost=c(0.001,0.01,0.1,1,10),
                        gamma=c(0.0001,0.001,0.01,0.1)))
summary(tune.out)
#cost 1 gamma 0.01


Parameter tuning of 'svm':

- sampling method: 10-fold cross validation 

- best parameters:
 cost gamma
    1   0.1

- best performance: 0.04166667 

- Detailed performance results:
    cost gamma      error dispersion
1  1e-03 1e-04 0.60000000 0.23174060
2  1e-02 1e-04 0.60000000 0.23174060
3  1e-01 1e-04 0.60000000 0.23174060
4  1e+00 1e-04 0.60000000 0.23174060
5  1e+01 1e-04 0.43333333 0.17916128
6  1e-03 1e-03 0.60000000 0.23174060
7  1e-02 1e-03 0.60000000 0.23174060
8  1e-01 1e-03 0.60000000 0.23174060
9  1e+00 1e-03 0.43333333 0.17916128
10 1e+01 1e-03 0.11666667 0.08050765
11 1e-03 1e-02 0.60000000 0.23174060
12 1e-02 1e-02 0.60000000 0.23174060
13 1e-01 1e-02 0.46666667 0.17213259
14 1e+00 1e-02 0.11666667 0.08050765
15 1e+01 1e-02 0.05000000 0.07027284
16 1e-03 1e-01 0.60000000 0.23174060
17 1e-02 1e-01 0.60000000 0.23174060
18 1e-01 1e-01 0.17500000 0.09170874
19 1e+00 1e-01 0.04166667 0.04392052
20 1e+01 1e-01 0.06666667 0.08606630


In [24]:
#실험을 통해 얻은 최적의 svm 모형
bestmodel<-tune.out$best.model
summary(bestmodel)


Call:
best.tune(method = svm, train.x = Result ~ ., data = train, ranges = list(cost = c(0.001, 
    0.01, 0.1, 1, 10), gamma = c(1e-04, 0.001, 0.01, 0.1)))


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  52

 ( 5 25 22 )


Number of Classes:  3 

Levels: 
 0 1 2




In [25]:
#학습용 데이터셋의 예측 정확도
pred<-predict(bestmodel,X_train)
table(y_train,pred)
mean(y_train==pred)

       pred
y_train  0  1  2
      0 40  0  0
      1  0 39  1
      2  0  1 39

[1] 0.9833333

In [26]:
#검증용 데이터셋의 예측 정확도
pred<-predict(bestmodel,X_test)
table(y_test,pred)
mean(y_test==pred)

      pred
y_test  0  1  2
     0 10  0  0
     1  0 10  0
     2  0  0 10

[1] 1